# Topic Model Data

This notebook uses MALLET to topic model project data. A full user guide with much more detail on configuring and running some of the cells below is available for this notebook in the <a href="README.md" target="_blank">README</a>. You may wish to consult this file to debug any issues that you run into.

### Info

__authors__    = 'Dan Baciu, Jeremy Douglass, Scott Kleinman, Lindsay Thomas'  
__copyright__ = 'copyright 2020, The WE1S Project'  
__license__   = 'GPL'  
__version__   = '2.0'  
__email__     = 'lindsaythomas@miami.edu'

## Settings

This cell sets up the workflow. Most users will run it as is.

In [ ]:
# Python imports
import os
from pathlib import Path
from IPython.display import display, HTML

# Define paths
current_dir       = %pwd
project_dir       = str(Path(current_dir).parent.parent)
json_dir          = project_dir + '/project_data/json'
model_dir         = project_dir + '/project_data/models'
import_file_path  = model_dir + '/doc_terms.txt'
log_file          = model_dir + '/mallet_import_log.txt'
language_model    = 'en_core_web_sm'
stoplist_file     = 'scripts/we1s_standard_stoplist.txt'

# Import scripts
%run scripts/prepare_mallet_import.py
%run scripts/mallet.py
%run scripts/scale_topics.py

# Display the project directory
display(HTML('<p><strong>Project Directory:</strong> ' + project_dir + '</p>'))

## Create File for Importing to MALLET

This cell extracts data from your project's json files (the documents in the project's `project_data/json` folder) and converts it to a single file to be imported into MALLET. For other methods of importing data (such as a directory of plain text files), see the **Import Data to MALLET** cell below.

By default, a preview of the first of rows of the import file is displayed with each row clipped at 200 characters. These settings can be adjusted in the `import.display()` call at the end of the cell. 

The conversion script first looks for a `bag_of_words` field in each json file. If none is present, the script generates a bag of words from the data in the `features` table. If that is not present, text in the `content` field is processed using spaCy, which can increase the processing time considerably, especially for large projects.

The `PrepareMalletImport` object can be initialised with additional settings to filter tokens according parts of speech or other criteria. For instance, `PrepareMalletImport(import_file_path, model_dir, language_model, stoplist_file, log_file, include_pos=['NOUN', 'VERB'])`. A full list of options is detailed in the <a href="README.md" target="_blank">README</a> file.

Finally, the file created by this cell **is not** the same as the `.mallet` file output via MALLET's `import-file` command. The file you are creating here is a text file used for feeding to the `import-file` command.

**Note:** You only need to run this cell once. If you run the notebook again, you can skip this cell.

In [ ]:
# Initialise the PrepareMalletImport object
prepare_import = PrepareMalletImport(import_file_path, model_dir, language_model, stoplist_file, log_file)

# Prepare data
prepare_import.prepare_data(json_dir)

# Display a preview of rows in doc_terms.txt file
display(HTML('<h4>Preview:</h4>'))
prepare_import.display(rows=5, clip=200)

## Setup MALLET

In the configuration setting below, list the number of topics for each model you wish to generate (note that multiple models can take a long time). List topic numbers in the format `[n1, n2, n3, etc.]`, where `n1`, `n2`, etc. are the numbers of topics you want to model.

The second cell in this section creates a `Mallet` object and prepares directories for your topic models.

In [ ]:
# Configure model topic numbers
num_topics = [] # E.g. [25, 50]

In [ ]:
# Create Mallet object
mallet = Mallet(num_topics, model_dir, import_file_path)

## Custom Configuration

Once the setup is complete,  you can change any of the MALLET settings below with commands like `mallet.optimize_interval = 11` (this model goes to 11!). You can use most arguments available in MALLET on the command line (see <a href="http://mallet.cs.umass.edu/topics.php" target="_blank">MALLET's documentation</a>), but replace hyphens with underscores. For example, reference MALLET's `--num-iterations` argument in this notebook with `mallet.num_iterations`.

One common use case for custom configurations is if you wish to import data from a directory of plain text files. You can do this with `mallet.import_source='path_to_directory'`. Please see the <a href="README.md" target="_blank">README</a> file for more information on importing plain text files. You can also choose MALLET's default tokenisation and stop words instead of the WE1S tokenisation algorithm and stoplist using `mallet.token_regex=None` and `mallet.remove_stopwords=True`. These configurations are provided for convenience in the cell below. You just have to uncomment them and run the cell.

In [ ]:
# Sample custom configuration
# mallet.import_source = 'path_to_directory'
# mallet.token_regex = None
# mallet.remove_stopwords = True

## Import Data to MALLET

This cell imports your data into MALLET.

If you wish to view the import command, create a new cell after this one and run `print(mallet.import_command)`.

In [ ]:
mallet.import_models()

## Train Models

By default, the models you configured in **Setup MALLET** above will be imported. If you wish to specify which models to import here, you can pass them to the training function as list in the cell below (e.g. `mallet.train_models([50, 100])`).

By default, progress is displayed in a progress bar. To monitor progress with text only, run `mallet.train_models(progress_bar=False)`. If you wish to display the MALLET's output instead of the progress monitor, run `mallet.train_models(display_output=True)`. If you wish to capture the output and display it at the end of processing, run `mallet.train_models(capture_output=True)`. You can also save the output to a log file with `mallet.train_models(log_file=log_file)`.

If you wish to view the train command, create a new cell after this one and run `print(mallet.train_command)`.

In [ ]:
mallet.train_models()

## Scale Topics (Optional)

This cell produces a CSV file with topic scaling metadata required by the Dfr-browser, pyLDAvis, and Topic Bubbles tools.

By default, scaling files will be generated for the models you configured in **Setup MALLET** above. If you wish to specify which models to scale here, replace `num_topics` with a list of topic desired topic numbers (e.g. `[50, 100]`) in the code below.

In [ ]:
# Get the model_vars from the configured number of topics
models = get_model_vars(num_topics, model_dir)
# Scale the models
scale(models, model_dir)